In [338]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
# ====================================================
# Library
# ====================================================
import sys

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
from torch.optim.optimizer import Optimizer

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau, LambdaLR

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from PIL import Image, ImageOps, ImageEnhance
from albumentations.core.transforms_interface import ImageOnlyTransform

import timm
from adamp import AdamP
from torch.cuda.amp import autocast, GradScaler
from warmup_scheduler import GradualWarmupScheduler
import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cpu')

import mlflow
remote_server_uri = "http://175.41.216.32:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment("seti-mizo")

In [339]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='eca_nfnet_l0'
    size=512
    scheduler='ReduceLROnPlateau' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    criterion = 'BCE'
    epochs=15 #15or20
    factor=0.2 # ReduceLROnPlateau
    patience=2 # ReduceLROnPlateau
    eps=1e-6 # ReduceLROnPlateau
    #T_max=15# CosineAnnealingLR
    #T_0=20 # CosineAnnealingWarmRestarts
    lr=5e-3
    min_lr=1e-6
    batch_size=128
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    mix_up=False # True or False
    cut_mix=False # True or False
    sam =True
    warmup_epochs=0
    multiplier=10
    multisample_dropout=False
    
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

In [340]:
train = pd.read_csv('meta_train.csv')
#train_old = pd.read_csv('seti-breakthrough-listen/old_leaky_data/train_labels_old.csv')

test = pd.read_csv('meta_test.csv')

def get_train_file_path(image_id):
    return "seti-breakthrough-listen/train/{}/{}.npy".format(image_id[0], image_id)
"""
def get_train__old_file_path(image_id):
    return "seti-breakthrough-listen/old_leaky_data/train_old/{}/{}.npy".format(image_id[0], image_id)
"""

def get_test_file_path(image_id):
    return "seti-breakthrough-listen/test/{}/{}.npy".format(image_id[0], image_id)

train['file_path'] = train['id'].apply(get_train_file_path)
test['file_path'] = test['id'].apply(get_test_file_path)

display(train.head())
display(test.head())

,id,target,file_path,fold,l0_pred,b0_ns_pred,b3_ns_pred,b3_ns_w_pred,v2_pred
0,000cd479c2106d4,0,seti-breakthrough-listen/train/0/000cd479c2106...,0,0.118882,0.052386,0.065241,0.043430,0.040911
1,00104dc954c2d30,0,seti-breakthrough-listen/train/0/00104dc954c2d...,0,0.007711,0.007562,0.009747,0.008566,0.005478
2,00154729d2fd371,0,seti-breakthrough-listen/train/0/00154729d2fd3...,0,0.029970,0.032488,0.018109,0.029937,0.020490
3,001f88b77b9b373,0,seti-breakthrough-listen/train/0/001f88b77b9b3...,0,0.092242,0.078834,0.105646,0.121659,0.113246
4,0021fc57e231440,0,seti-breakthrough-listen/train/0/0021fc57e2314...,0,0.065024,0.050495,0.049544,0.066693,0.055329


,id,l0_pred,b0_ns_pred,b3_ns_pred,b3_ns_w_pred,v2_pred,file_path
0,000bf832cae9ff1,0.057992,0.050963,0.055040,0.083506,0.048063,seti-breakthrough-listen/test/0/000bf832cae9ff...
1,000c74cc71a1140,0.062372,0.075789,0.060592,0.092812,0.046926,seti-breakthrough-listen/test/0/000c74cc71a114...
2,000f5f9851161d3,0.057835,0.055594,0.056416,0.076440,0.049687,seti-breakthrough-listen/test/0/000f5f9851161d...
3,000f7499e95aba6,0.088791,0.094974,0.097937,0.123739,0.113162,seti-breakthrough-listen/test/0/000f7499e95aba...
4,00133ce6ec257f9,0.062838,0.066846,0.058025,0.085500,0.046930,seti-breakthrough-listen/test/0/00133ce6ec257f...


In [341]:
x=np.array(train.drop(columns=['id','target','file_path','fold']))
print(x.shape)

(60000, 5)


In [342]:
x = x.reshape((len(train), -1))
print(x.shape)

(60000, 5)


In [343]:
x = x.reshape(60000, 1, 1, 5)
print(x.shape)

(60000, 1, 1, 5)


In [344]:
y = np.array(train['target'])
print(y.shape)

(60000,)


In [345]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [346]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [347]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [348]:
class Cnn2dClassifier(nn.Module):
    def __init__(self, num_feature, num_class):
        super(Cnn2dClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 128, kernel_size=(1,1), stride=1, padding=0)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=(1,1), stride=1, padding=0)

        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        
        self.layer_1 = nn.utils.weight_norm(nn.Linear(1280, 1280))
        self.dropout = nn.Dropout(p=0.5)
        
        self.fc = nn.Linear(1280, 1)

    def forward(self,x):

        x = self.conv1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        
        x = self.flatten(x)
        #print(x.shape)
 
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc(x)
        
        return x

In [349]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)
        
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [ ]:
print("Begin training.")
folds = StratifiedKFold(n_splits=5, shuffle=True, 
                        random_state=42).split(X, y)
total_acc=0.0
for fold, (trn_idx, val_idx) in enumerate(folds):
    accuracy_stats = { 'train': [], "val": [] }
    loss_stats = { 'train': [], "val": [] }
    best_val_acc = 0.0
    best_epoch = 0
    #if fold > 0:
    #    break 

    print('Training with FOLD{} started'.format(fold))
    
    X_train  = x[trn_idx]
    y_train  = y[trn_idx]
    
    X_val  = x[val_idx]
    y_val  = y[val_idx]  
    
    train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(),torch.from_numpy(y_train).float())
    val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(),torch.from_numpy(y_val).float())
    train_loader = DataLoader(dataset=train_dataset, batch_size=CFG.batch_size) #sampler=weighted_sampler
    val_loader = DataLoader(dataset=val_dataset, batch_size=CFG.batch_size)
    
    iters = len(train_loader)
    
    model = Cnn2dClassifier(num_feature = 5, num_class=1)
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    #optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
    optimizer = AdamP(model.parameters(), lr=7e-3, weight_decay=1e-6)
    #optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    #scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1, eta_min=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                       factor=0.75, patience=5, threshold=1e-5, threshold_mode='rel',
                                                       cooldown=3, min_lr=1e-8, eps=1e-08, verbose=False)
    optimizer.zero_grad()
    for e in tqdm(range(1, 51)):
        ## TRAINING
        train_epoch_loss = 0
        train_epoch_acc = 0
        timage_preds_all = []
        timage_targets_all = []
        
        model.train()
        for i, (X_train_batch, y_train_batch) in enumerate(train_loader):
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            
            # forward pass
            y_train_pred = model(X_train_batch)
            #print(y_train_pred)
            train_loss = criterion(y_train_pred.view(-1), y_train_batch)
            loss = train_loss
            train_acc = get_score(y_train_batch,y_train_pred.sigmoid().detach().cpu().numpy())
            timage_preds_all += [y_train_pred.sigmoid().detach().cpu().numpy()]
            timage_targets_all += [y_train_batch.detach().cpu().numpy()]
            # backward pass
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step(train_loss)
            scheduler.step(e + i / iters)

            # log results
            train_epoch_loss += loss.item()
            #train_epoch_acc += train_acc.item()
        
        timage_preds_all = np.concatenate(timage_preds_all)
        timage_targets_all = np.concatenate(timage_targets_all)
        #print('train accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
        
        ## VALIDATION    
        with torch.no_grad():
            val_epoch_loss = 0
            val_epoch_acc = 0
            model.eval()
            image_preds_all = []
            image_targets_all = []
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch,y_val_batch = X_val_batch.to(device), y_val_batch.to(device)    
                y_val_pred = model(X_val_batch)
                val_loss = criterion(y_val_pred.view(-1), y_val_batch)
                loss = val_loss             

                #val_acc = get_score(y_val_pred, y_val_batch)
                image_preds_all += [y_val_pred.sigmoid().to('cpu').numpy()]
                image_targets_all += [y_val_batch.detach().cpu().numpy()]
                
                # log results
                val_epoch_loss += loss.item()
                
        image_preds_all = np.concatenate(image_preds_all)
        image_targets_all = np.concatenate(image_targets_all)
        #print('validation accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))

        #scheduler.step(val_epoch_loss/len(val_loader))

        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        loss_stats['val'].append(val_epoch_loss/len(val_loader))
        accuracy_stats['train'].append((timage_preds_all==timage_targets_all).mean())
        accuracy_stats['val'].append((image_preds_all==image_targets_all).mean())

        if(image_preds_all==image_targets_all).mean() > best_val_acc:
            
            torch.save(model.state_dict(), f'MetaClassifier_{fold}_best.pth')
            best_val_acc = (image_preds_all==image_targets_all).mean()
            print(f'Save Model Acc:{best_val_acc:.5f} Epoch:{e}')
            best_epoch = e

        if (e % 5) == 0:
            print(f'Epoch {e+0:3}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} |\
            Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {(timage_preds_all==timage_targets_all).mean():.5f}|\
            Val Acc: {(image_preds_all==image_targets_all).mean():.5f}')
    total_acc += best_val_acc
    
print('Fold Ensemble Accuracy== {:.5f}'.format(total_acc/5))

Begin training.
Training with FOLD0 started


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch   5: | Train Loss: 0.15956 |            Val Loss: 0.15679 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  10: | Train Loss: 0.15931 |            Val Loss: 0.15678 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  15: | Train Loss: 0.15923 |            Val Loss: 0.15677 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  20: | Train Loss: 0.15947 |            Val Loss: 0.15677 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  25: | Train Loss: 0.15954 |            Val Loss: 0.15676 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  30: | Train Loss: 0.15925 |            Val Loss: 0.15676 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  35: | Train Loss: 0.15940 |            Val Loss: 0.15675 | Train Acc: 0.00000|            Val Acc: 0.00000
Epoch  40: | Train Loss: 0.15906 |            Val Loss: 0.15675 | Train Acc: 0.00000|            Val Acc: 0.00000
